In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from keras import layers
from gymnasium.wrappers import FrameStack
from gymnasium.experimental.wrappers import GrayscaleObservationV0
import gymnasium as gym
import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\shatchiv\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:423: UserWarning: WARN: Custom namespace `ALE` is being overridden by namespace `ALE`. If you are developing a plugin you shouldn't specify a namespace in `register` calls. The namespace is specified through the entry point package metadata.
  logger.warn(


In [2]:
env = gym.make('ALE/Frogger-v5') # Human render mode slows things down A LOT

In [3]:
env = GrayscaleObservationV0(env) # Remove RGB channels (make gray) in order to decrease amount of data to process
env.observation_space.shape

(210, 160)

In [4]:
seed = 42 # Allows us to repeat the same patterns of game play
env = FrameStack(env, 4) # Get 4 frames from game at a time
frames, width, height = env.observation_space.shape
env.seed(seed)

C:\Users\shatchiv\AppData\Roaming\Python\Python39\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.seed to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.seed` for environment variables or `env.get_wrapper_attr('seed')` that will search the reminding wrappers.
  logger.warn(


(3444837047, 2669555309)

In [5]:
# This network learns an approximation of the Q-table, which is a mapping between the states and actions that an agent will take. 
# For every state we'll have FIVE actions that can be taken (NOOP, Up, Down, Left, Right). 
# The environment provides the state, and the action is chosen by selecting the largest of the five Q-values predicted in the output layer of the CNN.

num_actions = 5

def create_CNN():
    
    # Change this!
    return keras.Sequential(
        [
            layers.Lambda(
                lambda tensor: keras.ops.transpose(tensor, [0, 2, 3, 1]),
                output_shape=(210, 160, 4),
                input_shape=(4, 210, 160),
            ),
            
            
            
            
            
            
            # Convolutions on the frames on the screen
            layers.Conv2D(32, 8, strides=4, activation="relu", input_shape=(4, 210, 160)),
            layers.Conv2D(64, 4, strides=2, activation="relu"),
            layers.Conv2D(64, 3, strides=6, activation="relu"),
            layers.Conv2D(64, 2, strides=4, activation="relu"),
            layers.Conv2D(64, 1, strides=2, activation="relu"),
            layers.Flatten(),
            layers.Dense(512, activation="relu"),
            layers.Dense(num_actions, activation="linear"),
        ]
    )

In [6]:
# The first neural net makes the predictions for Q-values, which are used to take an action.
cnn1 = create_CNN()

# A second cnn is used to predict future rewards. The weights of the second cnn get updated every 10000 steps.
cnn2 = create_CNN()

C:\Users\shatchiv\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\shatchiv\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
# Hyperparamaters DQN Algorithm

gamma = 0.99  # Discount factor in Bellman's equuation
epsilon = 1  # Epsilon greedy parameter for Q learning algorithm
max_steps_per_episode = 50 #Deepmind trained for "a total of 50 million frames (~38 days of game play)"
max_episodes = 10000  # Number of episodes you let the AI train. Keep above 1!
epsilon_min = 0.1  # Smallest epsilon value possible
epsilon_max = 1.0  # Largest epsilon value possible
epsilon_interval = (epsilon_max - epsilon_min)  # Rate we reduce chance of random action being taken (eventually, we don't want to take many random actions)

# Somre more important variables
batch_size = 32  # Size of sample taken from "replay buffer"
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0
# Maximum replay length
# Note: A Deepmind paper suggests 1000000, however this can cause memory issues
max_memory_length = 100000
# Train the model after 4 actions
update_after_actions = 4
# How often to update cnn2
update_cnn2 = 10000
# Using huber loss to check for convergance of Qs
loss_function = keras.losses.Huber()

In [ ]:
# DQN Algorithm
while True:
    observation, _ = env.reset()
    state = np.array(observation)
    episode_reward = 0
    
    for timestep in range(1, max_steps_per_episode):
        frame_count += 1
        # Use epsilon-greedy policy to explore or exploit
            # If current frame_count is less than 50,000 or epsilon is greater than a random number between 0 and 1
        print("Frame Count: ", frame_count)
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict Q-values from environment state
            state_tensor = keras.ops.convert_to_tensor(state) # Turn state (frame in game) into a Tensor Object (think matrix)
            state_tensor = keras.ops.expand_dims(state_tensor, 0) # Add to the current batch
            action_probs = cnn1(state_tensor, training=False)
            print(action_probs)
            # Take best action
            action = keras.ops.argmax(action_probs[0]).numpy()
        
        # Decrease probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)
        
        # Take action in environment
        print("Action: ", action)
        state_next, reward, done, _, _ = env.step(action)
        state_next = np.array(state_next)
        
        # Sum rewards across entire episode
        episode_reward += reward

        # Save actions and states in "replay buffer"
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next
        
        # Update every fourth frame AND once batch size is greater than 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = keras.ops.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states using cnn2
            future_rewards = cnn2.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward (Bellman's Equation)
            updated_q_values = rewards_sample + gamma * keras.ops.amax(future_rewards, axis=1)

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a 'mask' (matrix of 0s and 1s) so we only calculate loss on the updated Q-values
            masks = keras.ops.one_hot(action_sample, num_actions)
            
            # Train the cnn1 using updated Q-values
            with tf.GradientTape() as tape:
                q_values = cnn1(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = keras.ops.sum(keras.ops.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation - how cnn1 is updated
            grads = tape.gradient(loss, cnn1.trainable_variables)
            optimizer.apply_gradients(zip(grads, cnn1.trainable_variables))
        
        # Time to update cnn2?
        if frame_count % update_cnn2 == 0:
            # update cnn2 with new weights
            cnn2.set_weights(cnn1.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            #print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            print("DONE")
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    
    running_reward = np.mean(episode_reward_history)
    print("Average Reward Across All Episodes: ", running_reward)
    episode_count += 1

    if running_reward > 40:  # Condition to consider game "learned"
        print("Learned at episode {}!".format(episode_count))
        break

    if (max_episodes > 0 and episode_count >= max_episodes):  # Maximum number of episodes reached
        print("Stopped at episode {}!".format(episode_count))
        break

Frame Count:  1
Action:  0
Frame Count:  2
Action:  0
Frame Count:  3
Action:  4
Frame Count:  4
Action:  3
Frame Count:  5
Action:  3
Frame Count:  6
Action:  2
Frame Count:  7
Action:  1
Frame Count:  8
Action:  3
Frame Count:  9
Action:  1
Frame Count:  10
Action:  0
Frame Count:  11
Action:  4
Frame Count:  12
Action:  2
Frame Count:  13
Action:  2
Frame Count:  14
Action:  1
Frame Count:  15
Action:  3
Frame Count:  16
Action:  1
Frame Count:  17
Action:  4
Frame Count:  18
Action:  3
Frame Count:  19
Action:  2
Frame Count:  20
Action:  1
Frame Count:  21
Action:  2
Frame Count:  22
Action:  3
Frame Count:  23
Action:  1
Frame Count:  24
Action:  4
Frame Count:  25
Action:  3
Frame Count:  26
Action:  1
Frame Count:  27
Action:  2
Frame Count:  28
Action:  1
Frame Count:  29
Action:  0
Frame Count:  30
Action:  1
Frame Count:  31
Action:  1
Frame Count:  32
Action:  1
Frame Count:  33
Action:  2
Frame Count:  34
Action:  1
Frame Count:  35
Action:  2
Frame Count:  36
Action:  1
1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  189
Action:  3
Frame Count:  190
Action:  2
Frame Count:  191
Action:  2
Frame Count:  192
Action:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  193
Action:  3
Frame Count:  194
Action:  1
Frame Count:  195
Action:  1
Frame Count:  196
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Average Reward Across All Episodes:  0.0
Frame Count:  197
Action:  4
Frame Count:  198
Action:  1
Frame Count:  199
Action:  2
Frame Count:  200
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame Count:  201
Action:  4
Frame Count:  202
Action:  2
Frame Count:  203
Action:  2
Frame Count:  204
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Frame Count:  205
Action:  3
Frame Count:  206
Action:  4
Frame Count:  207
Action:  3
Frame Count:  208
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  209
Action:  2
Frame Count:  210
Action:  3
Frame Count:  211
Action:  0
Frame Count:  212
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/st

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Frame Count:  361
Action:  2
Frame Count:  362
Action:  0
Frame Count:  363
Action:  0
Frame Count:  364
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame Count:  365
Action:  0
Frame Count:  366
Action:  4
Frame Count:  367
Action:  0
Frame Count:  368
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  369
Action:  2
Frame Count:  370
Action:  3
Frame Count:  371
Action:  2
Frame Count:  372
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  373
Action:  2
Frame Count:  374
Action:  2
Frame Count:  375
Action:  4
Frame Count:  376
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Frame Count:  377
Action:  4
Frame Count:  378
Action:  4
Frame Count:  379
Action:  2
Frame Count:  380
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  381
Action:  3
Frame Count:  382
Action:  4
Frame Count:  383
Action:  1
Frame Count:  384
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  385
Action:  0
Frame Cou

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  533
Action:  4
Frame Count:  534
Action:  2
Frame Count:  535
Action:  1
Frame Count:  536
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  537
Action:  1
Frame Count:  538
Action:  1
Frame Count:  539
Action:  4
Average Reward Across All Episodes:  0.0
Frame Count:  540
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  541
Action:  4
Frame Count:  542
Action:  2
Frame Count:  543
Action:  4
Frame Count:  544
Action:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  545
Action:  0
Frame Count:  546
Action:  0
Frame Count:  547
Action:  0
Frame Count:  548
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  549
Action:  4
Frame Count:  550
Action:  4
Frame Count:  551
Action:  0
Frame Count:  552
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Frame Count:  553
Action:  2
Frame Count:  554
Action:  4
Frame Count:  555
Action:  2
Frame Count:  556
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/st

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  705
Action:  3
Frame Count:  706
Action:  1
Frame Count:  707
Action:  3
Frame Count:  708
Action:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame Count:  709
Action:  2
Frame Count:  710
Action:  3
Frame Count:  711
Action:  2
Frame Count:  712
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  713
Action:  1
Frame Count:  714
Action:  0
Frame Count:  715
Action:  4
Frame Count:  716
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  717
Action:  3
Frame Count:  718
Action:  0
Frame Count:  719
Action:  3
Frame Count:  720
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame Count:  721
Action:  3
Frame Count:  722
Action:  4
Frame Count:  723
Action:  1
Frame Count:  724
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame Count:  725
Action:  2
Frame Count:  726
Action:  4
Frame Count:  727
Action:  3
Frame Count:  728
Action:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Frame Count:  729
Action:  1
Frame Cou

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame Count:  7601
Action:  4
Frame Count:  7602
Action:  2
Frame Count:  7603
Action:  1
Frame Count:  7604
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  7605
Action:  4
Frame Count:  7606
Action:  1
Frame Count:  7607
Action:  3
Frame Count:  7608
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  7609
Action:  3
Frame Count:  7610
Action:  0
Frame Count:  7611
Action:  0
Frame Count:  7612
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  7613
Action:  1
Frame Count:  7614
Action:  1
Frame Count:  7615
Action:  0
Frame Count:  7616
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  7617
Action:  0
Frame Count:  7618
Action:  1
Frame Count:  7619
Action:  1
Frame Count:  7620
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  7621
Action:  1
Frame Count:  7622
Action:  4
Frame Count:  7623
Action:  2
Frame Count:  7624
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame Count:  7769
Action:  1
Frame Count:  7770
Action:  2
Frame Count:  7771
Action:  3
Frame Count:  7772
Action:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame Count:  7773
Action:  0
Frame Count:  7774
Action:  0
Frame Count:  7775
Action:  2
Frame Count:  7776
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Frame Count:  7777
Action:  1
Frame Count:  7778
Action:  3
Frame Count:  7779
Action:  1
Frame Count:  7780
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  7781
Action:  3
Frame Count:  7782
Action:  4
Frame Count:  7783
Action:  0
Frame Count:  7784
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Frame Count:  7785
Action:  3
Frame Count:  7786
Action:  4
Frame Count:  7787
Action:  2
Frame Count:  7788
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  7789
Action:  2
Frame Count:  7790
Action:  4
Frame Count:  7791
Action:  2
Average Reward Across All Episodes:  0.0
Frame Count:  7792
Action:  0
1/1 ━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  7937
Action:  1
Frame Count:  7938
Action:  0
Average Reward Across All Episodes:  0.0
Frame Count:  7939
Action:  1
Frame Count:  7940
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Frame Count:  7941
Action:  1
Frame Count:  7942
Action:  0
Frame Count:  7943
Action:  2
Frame Count:  7944
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  7945
Action:  0
Frame Count:  7946
Action:  1
Frame Count:  7947
Action:  2
Frame Count:  7948
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  7949
Action:  0
Frame Count:  7950
Action:  2
Frame Count:  7951
Action:  4
Frame Count:  7952
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  7953
Action:  4
Frame Count:  7954
Action:  3
Frame Count:  7955
Action:  2
Frame Count:  7956
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame Count:  7957
Action:  1
Frame Count:  7958
Action:  2
Frame Count:  7959
Action:  4
Frame Count:  7960
Action:  2
1/1 ━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Frame Count:  8105
Action:  4
Frame Count:  8106
Action:  3
Frame Count:  8107
Action:  3
Frame Count:  8108
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame Count:  8109
Action:  0
Frame Count:  8110
Action:  0
Frame Count:  8111
Action:  2
Frame Count:  8112
Action:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Frame Count:  8113
Action:  1
Frame Count:  8114
Action:  3
Frame Count:  8115
Action:  1
Frame Count:  8116
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Frame Count:  8117
Action:  3
Frame Count:  8118
Action:  2
Frame Count:  8119
Action:  4
Frame Count:  8120
Action:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Frame Count:  8121
Action:  0
Frame Count:  8122
Action:  4
Frame Count:  8123
Action:  2
Frame Count:  8124
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame Count:  8125
Action:  3
Frame Count:  8126
Action:  1
Frame Count:  8127
Action:  4
Frame Count:  8128
Action:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Frame Count:  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  8273
Action:  4
Frame Count:  8274
Action:  2
Frame Count:  8275
Action:  4
Frame Count:  8276
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Frame Count:  8277
Action:  1
Frame Count:  8278
Action:  3
Frame Count:  8279
Action:  4
Frame Count:  8280
Action:  4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  8281
Action:  4
Average Reward Across All Episodes:  0.0
Frame Count:  8282
Action:  4
Frame Count:  8283
Action:  4
Frame Count:  8284
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  8285
Action:  0
Frame Count:  8286
Action:  1
Frame Count:  8287
Action:  0
Frame Count:  8288
Action:  3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Frame Count:  8289
Action:  4
Frame Count:  8290
Action:  0
Frame Count:  8291
Action:  3
Frame Count:  8292
Action:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Frame Count:  8293
Action:  0
Frame Count:  8294
Action:  4
Frame Count:  8295
Action:  2
Frame Count:  8296
Action:  4
1/1 ━━━━━━━

In [ ]:
#cnn1.save_weights("cnn1v2.weights.h5") # Save the weights of each CNN
#cnn2.save_weights("cnn2v2.weights.h5")

In [ ]:
model = create_CNN()
model.load_weights('cnn1.weights.h5')
env = gym.make('ALE/Frogger-v5', render_mode="human")
env = GrayscaleObservationV0(env)
env = FrameStack(env, 4) # Get 4 frames from game at a time
frames, width, height = env.observation_space.shape
env.reset()
game_over = False

In [ ]:
state, reward, game_over, x, _ = env.step(0)
while not game_over:
    
    #Predict action using the trained model
    q_values = model.predict(np.expand_dims(state, axis=0))
    action = np.argmax(q_values)

    # Take action in the environment
    next_state, reward, game_over, x, _ = env.step(action)

    # Update current state
    state = next_state